# NBA API Issue #596 - ScoreboardV2 Deprecation Warning Demo

This notebook demonstrates the deprecation warning added to `ScoreboardV2` to guide users toward `ScoreboardV3`.

## Background

ScoreboardV2 has a known bug where it returns empty line scores for 2025-26 season games between October 22, 2025 and December 25, 2025. Investigation revealed:

- ❌ ScoreboardV2: Returns empty line scores for early 2025 season (Oct 22 - Dec 25)
- ✅ ScoreboardV3: Works correctly for ALL dates (100% backward compatible)

**Solution:** Add deprecation warning to ScoreboardV2 following library's established pattern.

See: https://github.com/swar/nba_api/issues/596

## Installation

Install the patched version from the branch:

In [ ]:
# Install from the fix branch
!pip install git+https://github.com/brandonhawi/nba_api@fix/scoreboard-v2-deprecation

## Demo 1: ScoreboardV2 Shows Deprecation Warning

Using ScoreboardV2 will now show a deprecation warning:

In [ ]:
import warnings

# Make sure warnings are visible
warnings.filterwarnings('default', category=DeprecationWarning)

from nba_api.stats.endpoints import ScoreboardV2

# This will trigger the deprecation warning
print("Creating ScoreboardV2 instance...\n")
scoreboard = ScoreboardV2(game_date='2025-10-22', timeout=60)

games = scoreboard.game_header.get_dict()["data"]
linescores = scoreboard.line_score.get_dict()["data"]

print(f"\nGames found: {len(games)}")
print(f"Line scores found: {len(linescores)}")

if len(games) > 0 and len(linescores) == 0:
    print("⚠️  Bug confirmed: Games exist but line scores are empty")

## Demo 2: ScoreboardV3 Works Without Warning

ScoreboardV3 is the recommended alternative:

In [ ]:
from nba_api.stats.endpoints import ScoreboardV3

# No deprecation warning with V3
print("Creating ScoreboardV3 instance...\n")
scoreboard_v3 = ScoreboardV3(game_date='2025-10-22', timeout=60)

# V3 returns games in a different structure
v3_data = scoreboard_v3.get_dict()
v3_games = v3_data.get("scoreboard", {}).get("games", [])
v3_linescores = scoreboard_v3.line_score.get_dict()["data"]

print(f"Games found: {len(v3_games)}")
print(f"Line scores found: {len(v3_linescores)}")
print("✅ ScoreboardV3 works correctly!")

## Demo 3: V3 is Backward Compatible

ScoreboardV3 works for historical seasons too:

In [ ]:
# Test with a 2024 season date
print("Testing ScoreboardV3 with 2024 season...\n")
scoreboard_2024 = ScoreboardV3(game_date='2024-10-22', timeout=60)

v3_data_2024 = scoreboard_2024.get_dict()
v3_games_2024 = v3_data_2024.get("scoreboard", {}).get("games", [])
v3_linescores_2024 = scoreboard_2024.line_score.get_dict()["data"]

print(f"2024 Games found: {len(v3_games_2024)}")
print(f"2024 Line scores found: {len(v3_linescores_2024)}")
print("✅ V3 is 100% backward compatible!")

## Summary

### The Deprecation Warning Pattern

The warning follows the established pattern in nba_api (same as BoxScoreTraditionalV2):

1. **Module docstring** - Documentation deprecation notice
2. **Class docstring** - Sphinx deprecation directive
3. **Runtime warning** - `DeprecationWarning` in `__init__`
4. **Clear migration path** - Recommends ScoreboardV3

### Migration Guide

```python
# Old (deprecated):
from nba_api.stats.endpoints import ScoreboardV2
scoreboard = ScoreboardV2(game_date='2025-10-22')

# New (recommended):
from nba_api.stats.endpoints import ScoreboardV3
scoreboard = ScoreboardV3(game_date='2025-10-22')
```

### Why This Matters

- ScoreboardV2 has a date-based bug (empty line scores Oct 22 - Dec 25, 2025)
- ScoreboardV3 works for all dates (tested back to 2020-21 season)
- The deprecation warning guides users to the working endpoint

**Full analysis:** [playground/final_bug_analysis.md](https://github.com/brandonhawi/playground/blob/master/final_bug_analysis.md)